In [4]:
import pandas as pd
import matplotlib.pyplot as plt

Title: "A/B Testing for Screen Guard Sales in Mobile Accessories Marketplace"

Description:
In this project, we delve into  A/B testing to optimize the sales strategy for screen guards in a mobile accessories marketplace. Our primary focus is on customers who are already inclined to purchase two items: mobile phones and back covers. The A/B testpurpose = obtain insights regarding the effectiveness of different campaign approaches in promoting screen guards.

Hypothesis:
The core question driving this analysis is whether the conversion rate (purchase_flag) is influenced by the specific suggestion made during the purchasing process. For instance, we investigate whether suggesting the purchase of a screen guard in conjunction with a mobile phone or a back cover affects the likelihood of a successful sale. 
The purchase flag becomes our key metric, taking a value of zero when the suggested combination does not result in a sale and one when the customer decides to make the purchase.

Objective:
Through analysis of the A/B test results, we aim to identify the more successful campaign approach. By understanding the correlation between the suggestion and the subsequent purchase behavior, we can refine our recommendation strategies to maximize conversion rates and enhance the overall sales performance of screen guards in tandem with mobile phones and back covers.

Methodology:
Our approach involves assessing the conversion rates for each campaign variant and drawing meaningful conclusions from the data. Utilizing statistical methods and visualization tools, we seek to provide actionable insights that can inform future marketing strategies and contribute to the ongoing optimization of screen guard sales in the mobile accessories marketplace.

This project is an example for data-driven decision-making in the realm of e-commerce.

In [5]:
filepath1 = r'C:..ndation_screencover.xlsx'

In [7]:
rec_data = pd.read_excel(filepath1)

In [8]:
rec_data.head(5)

,customer_id,Recommendation_name,Recommendation_date,Suggestion_type,Purchase_flag
0,1,Recommend_screenguard,2023-03-07,with phone,0
1,2,Recommend_screenguard,2023-03-08,with cover,0
2,3,Recommend_screenguard,2023-03-09,with cover,1
3,4,Recommend_screenguard,2023-03-10,with cover,1
4,5,Recommend_screenguard,2023-03-13,with phone,0


## keep only the first 5 columns:

In [76]:
# Keep only the first 5 columns
#rec_data_cleaned.iloc[:, :5] is used to select all rows (:) and the first 5 columns (:5).
r_data = rec_data.iloc[:, :5]

In [77]:
# Drop columns with all NaN values
#rec_data.dropna(axis=1, how='all')
# or, alternatively,     
#rec_data.dropna(inplace=True)

In [78]:
r_data.shape

(362, 5)

In [79]:
# Set display options to show all columns and rows without truncation
pd.set_option('display.max_columns', None)

In [80]:
# Check the resulting DataFrame
print(r_data.head())

   customer_id    Recommendation_name Recommendation_date Suggestion_type  \
0            1  Recommend_screenguard          2023-03-07      with phone   
1            2  Recommend_screenguard          2023-03-08      with cover   
2            3  Recommend_screenguard          2023-03-09      with cover   
3            4  Recommend_screenguard          2023-03-10      with cover   
4            5  Recommend_screenguard          2023-03-13      with phone   

   Purchase_flag  
0              0  
1              0  
2              1  
3              1  
4              0  


In [81]:
# Convert column names to lowercase
r_data.columns = r_data.columns.str.lower()

# Check the DataFrame with lowercase column names
print(r_data.head())

   customer_id    recommendation_name recommendation_date suggestion_type  \
0            1  Recommend_screenguard          2023-03-07      with phone   
1            2  Recommend_screenguard          2023-03-08      with cover   
2            3  Recommend_screenguard          2023-03-09      with cover   
3            4  Recommend_screenguard          2023-03-10      with cover   
4            5  Recommend_screenguard          2023-03-13      with phone   

   purchase_flag  
0              0  
1              0  
2              1  
3              1  
4              0  


In [82]:
r_data.groupby("suggestion_type")["purchase_flag"].value_counts()  ##the error is the white spaces

suggestion_type  purchase_flag
with cover       1                119
                 0                 91
with cover       1                  4
                 0                  1
with phone       0                 88
                 1                 59
Name: purchase_flag, dtype: int64

## REGEX needs to be applied here, to clean the trailing spaces and whitespaces

In [83]:
# In this code:  r'\s+' is the regular expression pattern that matches one or more whitespace characters.
# str.replace(r'\s+', '') is used to replace all occurrences of one or more whitespace characters with an empty string.
# apply is used to apply this replacement to each element in the 'suggestion_type' column.
# This will remove both leading and trailing spaces as well as any inner spaces within the strings in the 'suggestion_type' column.

r_data['suggestion_type'] = r_data['suggestion_type'].apply(lambda x: pd.Series(x).astype(str).str.replace(r'\s+', ''))


<ipython-input-83-071e6e2feadc>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  r_data['suggestion_type'] = r_data['suggestion_type'].apply(lambda x: pd.Series(x).astype(str).str.replace(r'\s+', ''))


In [84]:
r_data.groupby("suggestion_type")["purchase_flag"].value_counts()

#with cover, no purchase is 116  and when you recommended, with cover, purchase was 102
#which makes in total the recommended with cover to be  116 +102  out of this, how many times people bought the product? 102 times
#with phone you recommended 91+61, but people bought 61

suggestion_type  purchase_flag
withcover        1                123
                 0                 92
withphone        0                 88
                 1                 59
Name: purchase_flag, dtype: int64

In [85]:
## building the contingency table which relates to the labels 0 and 1, which is applicable in case of the two categorical variables
#we build this contingency table showing who bought and who didnt buy
CT_values = pd.crosstab(r_data["suggestion_type"], r_data["purchase_flag"]).values
CT_values

array([[ 92, 123],
       [ 88,  59]], dtype=int64)

## PURCHASE RATE, shows that there is a difference of 0.06 in the rates of selling, due to recommendations made

In [86]:
#once we have the contingency table, we can calculate the purchase rate:

with_phone_rate = 88 / (12 + 81)

with_cover_rate = 82 / (116 + 82)
print(with_phone_rate, with_cover_rate)

0.47093023255813954 0.41414141414141414


# CHI _ SQUARE TESTING  and HYPOTHESIS TESTING

In [ ]:
# Null Hypothesis: PURCHASE RATE IS ALWAYS THE SAME, REGARDLESS OF ANY RECCOMENDATION_TYPE YOU MIGHT HAVE MADE. The 2 variables  are independent.
# Alternative Hypothesis:THERE IS A RELATIONSHIP BETWEEN PURCHASE_RATE AND RECCOMENDATION _TYPE, they are not independent

In [67]:
from scipy.stats import chi2_contingency


In [68]:
chi2_statistic, p_value, dof, expected_values = chi2_contingency(CT_values, correction = False)
print(chi2_statistic, p_value)

1.6104776750309888 0.20442466871755294


In [ ]:
#The test shows whether there's a significant difference in the  behaviour of frequencies  or not

In [65]:
prob = 0.95
alpha = 1.0 - prob
print('significance = %.3f, p=%.3f' %(alpha, p_value))

if p_value <= alpha:
    print("Dependent, we can reject H0 safely")
else:
    print("Independent, no sufficient evidence to reject H0")

significance = 0.010, p=0.204
Independent, no sufficient evidence to reject H0


###  Meaning that making a reccomendation did help, purchase rate depends on this. NB: if we change the excel file, so that the p_value is less than alpha, the outcome of hypothesis testing changes.
